In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time
from IPython.display import HTML, display
from ipywidgets import interact, widgets, Layout, HBox, VBox
import heapq 

With the help of AI, I was able to finish this project under a time limited situation. 

In [4]:
# Create a graph of Florida cities with their coordinates
def create_florida_graph():
    G = nx.Graph()
    
    # Add major Florida cities with approximate lat/long coordinates
    cities = {
        "Miami": (25.7617, -80.1918),
        "Orlando": (28.5383, -81.3792),
        "Tampa": (27.9506, -82.4572),
        "Jacksonville": (30.3322, -81.6557),
        "Tallahassee": (30.4383, -84.2807),
        "Gainesville": (29.6516, -82.3248),
        "Fort Lauderdale": (26.1224, -80.1373),
        "West Palm Beach": (26.7153, -80.0534),
        "Fort Myers": (26.6406, -81.8723),
        "Pensacola": (30.4213, -87.2169),
        "Daytona Beach": (29.2108, -81.0228),
        "St. Petersburg": (27.7676, -82.6403),
        "Sarasota": (27.3364, -82.5307),
        "Naples": (26.1420, -81.7948),
        "Key West": (24.5551, -81.7800),
    }
    
    # Add cities as nodes with position attributes
    for city, coords in cities.items():
        G.add_node(city, pos=(coords[1], coords[0]))  # Note: x=long, y=lat
    
    # Add major highways as edges with distances in miles, maybe not accurate
    highways = [
        ("Miami", "Fort Lauderdale", 30),
        ("Fort Lauderdale", "West Palm Beach", 45),
        ("West Palm Beach", "Orlando", 170),
        ("Orlando", "Daytona Beach", 56),
        ("Daytona Beach", "Jacksonville", 95),
        ("Jacksonville", "Gainesville", 71),
        ("Gainesville", "Tallahassee", 150),
        ("Tallahassee", "Pensacola", 200),
        ("Orlando", "Tampa", 84),
        ("Tampa", "St. Petersburg", 25),
        ("Tampa", "Gainesville", 130),
        ("Orlando", "Gainesville", 120),
        ("St. Petersburg", "Sarasota", 35),
        ("Sarasota", "Fort Myers", 75),
        ("Fort Myers", "Naples", 40),
        ("Naples", "Miami", 115),
        ("Miami", "Key West", 160),
    ]
    
    for city1, city2, distance in highways:
        G.add_edge(city1, city2, weight=distance)
        
    return G, cities

In [5]:
def bfs_with_steps(G, source, target):
    """
    Breadth-First Search implementation that returns the path and exploration steps.
    
    Args:
        G (networkx.Graph): Graph containing nodes and edges
        source: Starting node
        target: Destination node
    
    Returns:
        tuple: (path, steps) where:
            - path is the list of nodes from source to target
            - steps is a list of (current_node, next_node, current_path) tuples showing exploration
    """
    if source == target:
        return [source], []
        
    visited = {source}
    queue = [(source, [source])]
    steps = []
    
    while queue:
        (vertex, path) = queue.pop(0)
        for neighbor in G.neighbors(vertex):
            if neighbor == target:
                final_path = path + [neighbor]
                steps.append((vertex, neighbor, final_path))
                return final_path, steps
                
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                queue.append((neighbor, new_path))
                steps.append((vertex, neighbor, None))
    
    # No path found
    return None, steps

def dijkstra_with_steps(G, source, target):
    """
    Dijkstra's algorithm implementation with step-by-step tracking and priority queue optimization.
    
    Args:
        G (networkx.Graph): Graph with weighted edges (requires 'weight' attribute)
        source: Starting node
        target: Destination node
    
    Returns:
        tuple: (path, steps) where:
            - path is the shortest path from source to target
            - steps is a list of (current_node, neighbor_node, None) tuples showing exploration
    """
    if source == target:
        return [source], []
    
    # Initialize data structures
    distances = {node: float('infinity') for node in G.nodes()}
    distances[source] = 0
    previous = {node: None for node in G.nodes()}
    
    # Initialize priority queue: (distance, node)
    pq = [(0, source)]
    steps = []
    visited = set()
    
    while pq:
        # Get the node with minimum distance
        current_distance, current = heapq.heappop(pq)
        
        # Skip if we've processed this node already
        if current in visited:
            continue
            
        visited.add(current)
        
        # Early termination if target is reached
        if current == target:
            break
            
        # Skip unreachable nodes
        if current_distance == float('infinity'):
            break
            
        # Check all neighbors
        for neighbor in G.neighbors(current):
            if neighbor in visited:
                continue
                
            weight = G[current][neighbor]['weight']
            distance = distances[current] + weight
            
            # If we found a better path
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current
                heapq.heappush(pq, (distance, neighbor))
                steps.append((current, neighbor, None))
    
    # Reconstruct path if target was reached
    if previous[target] is not None or source == target:
        path = []
        current = target
        while current:
            path.append(current)
            current = previous[current]
        return path[::-1], steps
    
    # No path found
    return None, steps

def astar_with_steps(G, source, target):
    """
    A* algorithm implementation with step tracking and priority queue optimization.
    
    Args:
        G (networkx.Graph): Graph with weighted edges and node positions
        source: Starting node
        target: Destination node
    
    Returns:
        tuple: (path, steps) where:
            - path is the shortest path from source to target
            - steps is a list of exploration steps
    """
    if source == target:
        return [source], []
    
    # Get node positions for heuristic calculation
    pos = nx.get_node_attributes(G, 'pos')
    
    # Calculate heuristic (straight-line distance)
    def heuristic(n1, n2):
        x1, y1 = pos[n1]
        x2, y2 = pos[n2]
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2) * 70  # Rough miles conversion
    
    # Initialize data structures
    g_score = {node: float('infinity') for node in G.nodes()}
    g_score[source] = 0
    
    f_score = {node: float('infinity') for node in G.nodes()}
    f_score[source] = heuristic(source, target)
    
    # Use priority queue: (f_score, node_id, node) 
    # node_id is used to break ties consistently
    open_set = [(f_score[source], 0, source)]
    node_counter = 1  # For unique IDs in the priority queue
    
    closed_set = set()
    previous = {}
    steps = []
    
    while open_set:
        # Get node with minimum f_score
        _, _, current = heapq.heappop(open_set)
        
        # Early termination when target is reached
        if current == target:
            break
            
        if current in closed_set:
            continue
            
        closed_set.add(current)
        
        for neighbor in G.neighbors(current):
            if neighbor in closed_set:
                continue
                
            tentative_g_score = g_score[current] + G[current][neighbor]['weight']
            
            if tentative_g_score < g_score[neighbor]:
                # Found a better path to neighbor
                previous[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, target)
                
                # Add to priority queue
                heapq.heappush(open_set, (f_score[neighbor], node_counter, neighbor))
                node_counter += 1
                
                steps.append((current, neighbor, None))
    
    # Reconstruct path if target was reached
    if target in previous or source == target:
        path = []
        current = target
        while current in previous:
            path.append(current)
            current = previous[current]
        path.append(source)
        return path[::-1], steps
    
    # No path found
    return None, steps

In [51]:
def create_reliable_visualizer():
    """
    A implementation of the interactive visualizer that ensures
    the visualization updates properly when parameters change.
    """
    # Create the Florida graph
    G, cities = create_florida_graph()
    pos = nx.get_node_attributes(G, 'pos')
    
    
    # Create widgets
    algorithm_options = [
        ('BFS (Fewest cities/connections)', 'bfs'), 
        ('Dijkstra (Shortest total distance)', 'dijkstra'), 
        ('A* (Heuristic-guided shortest distance)', 'astar')
    ]
    algorithm_widget = widgets.Dropdown(
        options=algorithm_options,
        value='dijkstra',
        description='Algorithm:',
        style={'description_width': 'initial'}
    )
    
    city_list = list(cities.keys())
    start_widget = widgets.Dropdown(
        options=city_list,
        value='Tampa',
        description='Start City:',
        style={'description_width': 'initial'}
    )
    
    end_widget = widgets.Dropdown(
        options=city_list,
        value='Miami',
        description='End City:',
        style={'description_width': 'initial'}
    )
    
    speed_widget = widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=3.0,
        step=0.1,
        description='Speed:',
        style={'description_width': 'initial'}
    )
    
    numbers_widget = widgets.Checkbox(
        value=True,
        description='Show exploration order',
        style={'description_width': 'initial'}
    )
    
    # Create output areas
    output_text = widgets.Output()
    output_viz = widgets.Output()
    
    # Function to run when parameters change
    def run_visualization(algorithm, start, end, speed, show_numbers):
        with output_text:
            output_text.clear_output(wait=True)
            print(f"Running {algorithm} from {start} to {end}...")
            
            # Run the selected algorithm
            start_time = time.time()
            
            if algorithm == 'bfs':
                path, steps = bfs_with_steps(G, start, end)
                algorithm_name = "Breadth-First Search (BFS)"
                color = "purple"
            elif algorithm == 'dijkstra':
                path, steps = dijkstra_with_steps(G, start, end)
                algorithm_name = "Dijkstra's Algorithm"
                color = "blue"
            else:  # A*
                path, steps = astar_with_steps(G, start, end)
                algorithm_name = "A* Search Algorithm"
                color = "orange"
            
            execution_time = (time.time() - start_time) * 1000  # ms
            
            # Calculate stats
            if path:
                path_length = len(path) - 1
                path_distance = sum(G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
                
                # Display results
                print(f"Algorithm: {algorithm_name}")
                print(f"Path: {' -> '.join(path)}")
                print(f"Number of steps: {path_length}")
                print(f"Total distance: {path_distance} miles")
                print(f"Nodes explored: {len(steps)}")
                print(f"Execution time: {execution_time:.2f} ms")
            else:
                print(f"No path found from {start} to {end}!")
        
        with output_viz:
            output_viz.clear_output(wait=True)
            if path:
                # Create a static visualization that works reliably
                create_static_visualization(G, path, steps, start, end, algorithm_name, color, show_numbers)
    
    def create_static_visualization(G, path, steps, start, end, algorithm_name, color, show_numbers):
        """Create a static visualization of the algorithm result"""
        fig, ax = plt.subplots(figsize=(12, 10))
        pos = nx.get_node_attributes(G, 'pos')
        
        # Extract visited nodes from steps
        visited_nodes = set()
        node_order = {}
        counter = 0
        
        for current, neighbor, _ in steps:
            if current:
                visited_nodes.add(current)
                if current not in node_order:
                    counter += 1
                    node_order[current] = counter
                    
            if neighbor:
                visited_nodes.add(neighbor)
                if neighbor not in node_order:
                    counter += 1
                    node_order[neighbor] = counter
        
        # Draw edges
        nx.draw_networkx_edges(G, pos, ax=ax, edge_color='lightgray', width=1.0)
        
        # Draw edge weights
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, ax=ax)
        
        # Draw explored edges
        explored_edges = []
        for current, neighbor, _ in steps:
            if current and neighbor and G.has_edge(current, neighbor):
                explored_edges.append((current, neighbor))
                
        if explored_edges:
            nx.draw_networkx_edges(G, pos, edgelist=explored_edges, 
                                 edge_color=color, width=1.5, alpha=0.5, ax=ax)
        
        # Draw final path
        if path and len(path) > 1:
            path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            nx.draw_networkx_edges(G, pos, edgelist=path_edges,
                                 edge_color='red', width=3.0, ax=ax)
        
        # Prepare node colors based on status
        node_colors = []
        node_sizes = []
        
        for node in G.nodes():
            if node == start:  # Start node
                node_colors.append('green')
                node_sizes.append(600)
            elif node == end:  # End node
                node_colors.append('purple')
                node_sizes.append(600)
            elif node in path:  # Path node - RED as requested
                node_colors.append('red')
                node_sizes.append(550)
            elif node in visited_nodes:  # Explored node - YELLOW as requested
                node_colors.append('yellow')
                node_sizes.append(500)
            else:  # Unexplored
                node_colors.append('lightgray')
                node_sizes.append(450)
        
        # Draw nodes
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, ax=ax)
        nx.draw_networkx_labels(G, pos, font_weight='bold', ax=ax)
        
        # Draw exploration order numbers if enabled
        if show_numbers:
            for node, order in node_order.items():
                x, y = pos[node]
                # Position the order number above and to the right of the node
                # instead of directly on top of it
                offset_x = 0
                offset_y = 0.1
                
                circle = plt.Circle((x + offset_x, y + offset_y), 0.015, 
                                   fc='white', ec='black', alpha=0.9, zorder=10)
                ax.add_patch(circle)
                ax.text(x + offset_x, y + offset_y, str(order), 
                       ha='center', va='center', fontsize=8, 
                       fontweight='bold', zorder=11)
        
        # Set title and disable axis
        path_length = len(path) - 1 if path else 0
        path_distance = sum(G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1)) if path_length > 0 else 0
        title = f"{algorithm_name}: {start} to {end}\n"
        title += f"Distance: {path_distance} miles | Cities visited: {len(visited_nodes)}/{len(G.nodes())}"
        
        ax.set_title(title)
        ax.axis('off')
        plt.tight_layout()
        plt.show()
    
    # Create interactive function
    interactive_visualizer = widgets.interactive(
        run_visualization,
        algorithm=algorithm_widget,
        start=start_widget,
        end=end_widget,
        speed=speed_widget,
        show_numbers=numbers_widget
    )
    
    # Layout widgets and outputs
    layout = widgets.VBox([
        widgets.HBox([algorithm_widget]),
        widgets.HBox([start_widget, end_widget]),
        widgets.HBox([speed_widget, numbers_widget]),
        widgets.HTML(value="<small><i>Note: Changes to any parameter will automatically update the visualization</i></small>"),
        output_text,
        output_viz
    ])
    
    display(layout)
    
    # Run initial visualization
    run_visualization(algorithm_widget.value, start_widget.value, end_widget.value, 
                     speed_widget.value, numbers_widget.value)

In [53]:
# Execute the main function to create the interactive visualizer
def main():
    print("Florida Pathfinding Visualizer")
    print("=============================")
    print("This interactive tool demonstrates how different pathfinding algorithms")
    print("explore a graph representation of major Florida cities and highways.")
    print("\nSelect an algorithm, start city, end city, and visualization speed,")
    print("\nSpeed determines the visualization speed,")
    print("\nAlgorithms available:")
    print("- BFS: Finds the path with fewest edges (ignores distances)")
    print("- Dijkstra: Finds the shortest path by distance")
    print("- A*: Uses heuristics to efficiently find the shortest path")
    
    create_reliable_visualizer()

# Run the main function
main()

Florida Pathfinding Visualizer
This interactive tool demonstrates how different pathfinding algorithms
explore a graph representation of major Florida cities and highways.

Select an algorithm, start city, end city, and visualization speed,

Speed determines the visualization speed,

Algorithms available:
- BFS: Finds the path with fewest edges (ignores distances)
- Dijkstra: Finds the shortest path by distance
- A*: Uses heuristics to efficiently find the shortest path
